# MCP Client with Microsoft Agent Framework

In [ ]:
!pip install agent-framework --pre

In [ ]:
from agent_framework import ChatAgent, MCPStdioTool, MCPStreamableHTTPTool, MCPWebsocketTool
from agent_framework.openai import OpenAIChatClient

from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment_name = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    api_key=api_key,
 )

# MCP 接続先 (必要なら環境変数で上書き)
MCP_URL = os.getenv("MCP_URL", "http://localhost:8000/mcp")

def make_http_mcp(token: str) -> MCPStreamableHTTPTool:
    return MCPStreamableHTTPTool(
        name="http_mcp",
        url=MCP_URL,
        headers={"Authorization": f"Bearer {token}"},
    )

async def run_mcp_prompt(prompt: str, *, token: str, list_tools: bool = False) -> str:
    """
    Notebook/Jupyter のタスク境界を跨がないよう、
    ツール/エージェントはこの関数内で async with して都度開閉する。
    """
    async with make_http_mcp(token) as mcp_server, ChatAgent(
        name="assistant",
        chat_client=client,
        instructions="あなたは便利なアシスタントです。ユーザーの質問に答えるために、必要に応じて以下のツールを使用してください。",
    ) as agent:
        if list_tools:
            print(f"接続成功！利用可能なツール: {len(mcp_server.functions)}")
            for func in mcp_server.functions:
                print(f"  - {func.name}")
        result = await agent.run(prompt, tools=mcp_server)
        return result.text

In [ ]:
from azure.identity import DefaultAzureCredential
# Azure AI Services用のトークンを取得（audience: https://ai.azure.com）
credential = DefaultAzureCredential()
token = credential.get_token("https://storage.azure.com/.default")

access_token = token.token

# print(f"Access token acquired (expires in {token.expires_on})")
print(access_token)

In [ ]:
# MCPサーバーへの接続と実行（同一セル内で open/close する）
try:
    print("MCPサーバーへの接続を試みています...")
    text = await run_mcp_prompt(
        "ファイルを取得したい",
        token=access_token,
        list_tools=True,
    )
    print(text)
    connection_ok = True
except Exception as e:
    connection_ok = False
    print(f"接続エラー: {type(e).__name__}: {e}")
    print("\nMCPサーバーに接続できません。以下を確認してください:")
    print("1. サーバーが起動しているか: http://localhost:8000/mcp")
    print("2. ネットワーク接続が正常か")
    print("3. ファイアウォールの設定")

In [ ]:
result_text = await run_mcp_prompt("ユーザー情報を取得してください。", token=access_token)
print(result_text)

In [ ]:
result_text = await run_mcp_prompt("コンテンツ事業に関するＱ＆Ａ.pdfを取得してください。", token=access_token)
print(result_text)
